In [1]:
from data import update_game
from model.model import update_models, update_model_scores

In [2]:
#update_models()

Updating models
SVR (radial basis)...
Ridge...
XGB...
Stacking Models (like legos)
Done.


In [2]:
update_model_scores()

In [9]:
test_url = 'https://www.metacritic.com/game/tactical-breach-wizards/'
update_game(metacritic_url=test_url, title=None, classic=False)

https://www.metacritic.com/game/tactical-breach-wizards/ None
https://www.metacritic.com/game/tactical-breach-wizards/ Tactical Breach Wizards


Title          Tactical Breach Wizards
My Rating                          NaN
raw_score                     3.856536
model_score                   3.856536
Name: 559, dtype: object